In [1]:
import pandas as pd
import numpy as np

In [ ]:
def scheduleMix(schedules: list[np.ndarray], group: list[int]) -> list[np.ndarray]: 
    return np.sum(schedules[group], axis = 0)


In [ ]:

class event:
    scheduled = False
    def __init__(self, eventLength: int, startTime : list, endTime: list, group: np.ndarray, everybodyNeeded: bool): #event length is stored in terms of how many time interval blocks it takes.
        """ 
        eventLength is stored as an int being the amount of time blocks it would take up
        start Time and end time need some work as how they will be stored is still in question
        """
        self.eventLength = eventLength
        self.startTime = startTime
        self.endTime = endTime
        self.group = group
        self.everybodyNeeded = everybodyNeeded

    def scheduleEvent(self, schedules: np.ndarray , time: list, cancelIfEverybodyNeeded: bool):
        "Time is stored as a 3d cordinate being (day of the week, time of day by block #)"
        if cancelIfEverybodyNeeded:
            scheduleMix(schedules, self.group)
            for idx in range(self.eventLength): # Asummes 1 means the spot is filled already
                if not schedules[idx]:
                    return
        for member in self.group:
            for idx in range(self.eventLength): 
                if not schedules[(member,idx)]:
                    schedules[(member,idx)] = 1
        self.scheduled = True
                


                

    

In [ ]:
def algorithe(events: event, schedules: np.ndarray):
    eventMixes = [scheduleMix(schedules, oneEvent.group) for oneEvent in events]
    everbodyEvents = [oneEvent for oneEvent in events if oneEvent.everybodyNeeded]
    for everbodyEvent in everbodyEvents:
        

    


SyntaxError: invalid syntax (3248419516.py, line 1)

Experimental code below


In [ ]:
from perlin_noise import PerlinNoise

def generate_perlin_noise(shape, scale=10, threshold=0):
    noise = PerlinNoise(octaves=4)  
    noise_array = np.zeros(shape)
    
    x = np.linspace(0, 1, shape[0]) 
    y = np.linspace(0, 1, shape[1]) 
    xv, yv = np.meshgrid(x, y, indexing='ij')  
    
    for i in range(shape[0]):
        for j in range(shape[1]):
            noise_value = noise([xv[i, j] * scale, yv[i, j] * scale])
            noise_array[i, j] = 1 if noise_value > threshold else 0
    
    return noise_array

exampleSchedule = np.asarray([generate_perlin_noise((7, 135), scale=5, threshold=0) for _ in range(15)])   ################## Generates 15 example schedules to use for testing

groups = [np.asarray([5,7,9,1,11,13]),np.asarray([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])] ###################### define groups

print(exampleSchedule[0])

In [24]:
scheduleMix(exampleSchedule, groups[0])

array([[0., 0., 1., 1., 1., 2., 3., 3., 3., 1., 1., 1., 1., 2., 4., 4.,
        4., 3., 3., 2., 2., 4., 4., 4., 2., 3., 4., 2., 2., 3., 3., 3.,
        2., 2., 4., 4., 2., 3., 3., 3., 2., 4., 3., 2., 2., 3., 3., 3.,
        3., 3., 3., 4., 3., 4., 2., 2., 2., 2., 2., 2., 2., 4., 5., 5.,
        5., 5., 5., 0., 1., 1., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3.,
        2., 3., 3., 3., 2., 1., 2., 2., 4., 4., 4., 4., 3., 2., 4., 4.,
        2., 2., 3., 4., 3., 3., 3., 3., 3., 2., 1., 1., 5., 5., 4., 4.,
        4., 4., 2., 2., 2., 3., 5., 4., 4., 2., 2., 2., 4., 4., 4., 4.,
        2., 2., 2., 1., 1., 0., 0.],
       [1., 2., 3., 4., 3., 3., 3., 4., 4., 3., 3., 3., 2., 3., 2., 2.,
        1., 1., 1., 0., 0., 3., 3., 2., 3., 2., 2., 1., 2., 2., 3., 3.,
        3., 3., 3., 3., 3., 3., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.,
        2., 2., 2., 2., 3., 4., 5., 6., 5., 3., 2., 2., 2., 1., 4., 4.,
        4., 4., 4., 4., 4., 5., 4., 3., 3., 3., 4., 4., 4., 4., 3., 3.,
        3., 4., 3., 3., 3.,